## Setting up Environment

In [ ]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!kaggle datasets download -d kmader/skin-cancer-mnist-ham10000

Dataset URL: https://www.kaggle.com/datasets/kmader/skin-cancer-mnist-ham10000
License(s): CC-BY-NC-SA-4.0
100% 5.20G/5.20G [03:19<00:00, 28.1MB/s]
100% 5.20G/5.20G [03:19<00:00, 28.0MB/s]


In [ ]:
! mkdir data
! unzip -q skin-cancer-mnist-ham10000.zip -d data

In [ ]:
! mkdir models

### Import Libraries

In [ ]:
# Import Libraries
import os
import logging
import time
import pickle
import multiprocessing
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from PIL import Image
from collections import defaultdict

import albumentations as A
from albumentations.pytorch import ToTensorV2

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.amp import GradScaler, autocast
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
from torchvision.models import ResNet34_Weights
from torchvision import models

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import confusion_matrix

from transformers import AdamW, get_linear_schedule_with_warmup


# Set Device
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {DEVICE}")

print("Number of CPUs:", multiprocessing.cpu_count())

torch.backends.cudnn.benchmark = True


Using device: cuda
Number of CPUs: 8


In [ ]:
# Constants

DATA_DIR = 'data/'
#DATA_DIR = '/content/drive/MyDrive/data'

METADATA_FILE = os.path.join(DATA_DIR, 'HAM10000_metadata.csv')
IMAGE_DIRS = [os.path.join(DATA_DIR, 'HAM10000_images_part_1'),
              os.path.join(DATA_DIR, 'HAM10000_images_part_2')]


# Load Metadata
metadata = pd.read_csv(METADATA_FILE)
metadata.head()



,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear


In [ ]:
print(f"PyTorch Version: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"CUDA Version: {torch.version.cuda}")
    print(f"cuDNN Version: {torch.backends.cudnn.version()}")
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"GPU Count: {torch.cuda.device_count()}")

PyTorch Version: 2.4.1+cu121
CUDA Available: True
CUDA Version: 12.1
cuDNN Version: 90100
GPU Name: Tesla T4
GPU Count: 1


### Directories

In [ ]:
# Base directories for all cross-validation processes
BASE_RESULT_DIR = 'drive/MyDrive/fusion_Result'
BASE_MODEL_DIR = 'drive/MyDrive/fusion_Model'
BASE_CHECKPOINT_DIR = 'drive/MyDrive/fusion_Checkpoints'

# Nested Cross-Validation Directories
NESTED_RESULT_SAVE_DIR = os.path.join(BASE_RESULT_DIR, 'nested_cv')
NESTED_MODEL_SAVE_DIR = os.path.join(BASE_MODEL_DIR, 'nested_cv')
NESTED_CHECKPOINT_DIR = os.path.join(BASE_CHECKPOINT_DIR, 'nested_cv')

# 20-Fold Cross-Validation Directories
FOLD20_RESULT_SAVE_DIR = os.path.join(BASE_RESULT_DIR, 'fold20_cv')
FOLD20_MODEL_SAVE_DIR = os.path.join(BASE_MODEL_DIR, 'fold20_cv')
FOLD20_CHECKPOINT_DIR = os.path.join(BASE_CHECKPOINT_DIR, 'fold20_cv')
INDICES_SAVE_PATH = 'drive/MyDrive/fusion_indices.pkl'

## Setting up Custom Function


### StratifiedKFold


In [ ]:
class customStratifiedKFold:
    def __init__(self, n_split, shuffle=True, random_state=None):
        self.n_split = n_split
        self.shuffle = shuffle
        self.random_state = random_state

    def split(self, X, y):
        if self.shuffle and self.random_state is not None:
            np.random.seed(self.random_state)

        idx = np.arange(len(y))
        if self.shuffle:
            np.random.shuffle(idx)

        y_idx = defaultdict(list)
        for i, label in zip(idx, y):
            y_idx[label].append(i)

        splits = [[] for _ in range(self.n_split)]

        for label, indices in y_idx.items():
            if self.shuffle:
                np.random.shuffle(indices)
            split_portions = [len(indices) // self.n_split] * self.n_split
            for j in range(len(indices) % self.n_split):
                split_portions[j] += 1

            first = 0
            for k in range(self.n_split):
                last = first + split_portions[k]
                splits[k].extend(indices[first:last])
                first = last

        for i in range(self.n_split):
            test_idx = splits[i]
            train_idx = np.concatenate([splits[j] for j in range(self.n_split) if j != i])
            yield train_idx, test_idx

    def get_n_splits(self):
        return self.n_split

### Nested CV

In [ ]:
import os
import json
from collections import defaultdict
from tqdm import tqdm
import numpy as np
import pandas as pd
import torch
from torch.cuda.amp import GradScaler
from torch.utils.data import DataLoader
from torch import nn
from transformers import get_linear_schedule_with_warmup

def cross_validate(model_class, X, y, outer_splits=5, inner_splits=3, hypers=None,
                  metadata_features=None, image_dirs=None, feature_extractor=None,
                  transform_train=None, transform_val=None, device='cuda',
                  log_file='fusion_Checkpoints/nested_cv_log.txt'):
    """
    Performs nested cross-validation for hyperparameter tuning and evaluation with checkpointing.

    Args:
        model_class (nn.Module): The neural network model class to instantiate.
        X (pd.DataFrame): Feature DataFrame.
        y (np.ndarray): Labels.
        outer_splits (int): Number of outer folds.
        inner_splits (int): Number of inner folds for hyperparameter tuning.
        hypers (list): List of hyperparameter dictionaries.
        metadata_features (list): List of metadata feature column names.
        image_dirs (list): List of directories where images are stored.
        feature_extractor (ViTFeatureExtractor): Feature extractor for processing images.
        transform_train (callable): Transformations for training data.
        transform_val (callable): Transformations for validation data.
        device (str): Device to perform training on ('cuda' or 'cpu').
        log_file (str): Path to the log file.

    Returns:
        list: Results containing best hyperparameters and their scores for each outer fold.
    """
    outer_skf = customStratifiedKFold(n_split=outer_splits, shuffle=True, random_state=42)
    results = []

    splits_save_path = os.path.join(NESTED_CHECKPOINT_DIR, 'nested_cv_splits.json')
    progress_save_path = os.path.join(NESTED_CHECKPOINT_DIR, 'nested_cv_progress.json')

    if os.path.exists(progress_save_path):
        with open(progress_save_path, 'r') as f:
            progress = json.load(f)
        print("Loaded existing progress.")
    else:
        progress = {
            'completed_outer_folds': [],
            'outer_folds': {}
        }
        print("Initialized new progress tracking.")

    if os.path.exists(splits_save_path):
        with open(splits_save_path, 'r') as f:
            splits = json.load(f)

        splits = [(np.array(train), np.array(test)) for train, test in splits]
        print("Loaded existing data splits.")
    else:
        splits = list(outer_skf.split(X, y))

        splits_serializable = [(np.array(train).tolist(), np.array(test).tolist()) for train, test in splits]
        with open(splits_save_path, 'w') as f:
            json.dump(splits_serializable, f)
        print("Created and saved new data splits.")

    total_combinations = outer_splits * inner_splits * len(hypers)
    pbar = tqdm(total=total_combinations + outer_splits,
                desc="Nested Cross-Validation Progress",
                unit="combo",
                mininterval=5.0)

    with open(log_file, "a") as log:
        log.write("Cross-Validation Log\n")
        log.write("====================\n")

    for outer_fold_idx, (train_val_idx, test_idx) in enumerate(splits, 1):
        if outer_fold_idx in progress['completed_outer_folds']:
            print(f"Outer Fold {outer_fold_idx} already completed. Skipping.")
            pbar.update(1)
            continue

        print(f"\n=== Starting Outer Fold {outer_fold_idx}/{outer_splits} ===")
        X_train_val = X.iloc[train_val_idx].reset_index(drop=True)
        y_train_val = np.array(y)[train_val_idx]
        X_test = X.iloc[test_idx].reset_index(drop=True)
        y_test = np.array(y)[test_idx]

        if str(outer_fold_idx) not in progress['outer_folds']:
            progress['outer_folds'][str(outer_fold_idx)] = {
                'completed_inner_folds': {},
                'best_hyperparams': {},
                'best_score': 0.0
            }

        inner_skf = customStratifiedKFold(n_split=inner_splits, shuffle=True, random_state=42)
        inner_splits_list = list(inner_skf.split(X_train_val, y_train_val))

        for inner_fold_idx, (train_idx, val_idx) in enumerate(inner_splits_list, 1):
            inner_fold_key = str(inner_fold_idx)
            if inner_fold_key not in progress['outer_folds'][str(outer_fold_idx)]['completed_inner_folds']:
                progress['outer_folds'][str(outer_fold_idx)]['completed_inner_folds'][inner_fold_key] = {
                    'completed_hyperparams': []
                }

            print(f"\n--- Outer Fold {outer_fold_idx}, Inner Fold {inner_fold_idx}/{inner_splits} ---")

            X_train = X_train_val.iloc[train_idx].reset_index(drop=True)
            y_train = np.array(y_train_val)[train_idx]
            X_val = X_train_val.iloc[val_idx].reset_index(drop=True)
            y_val = np.array(y_train_val)[val_idx]

            for hyper_idx, hyper in enumerate(hypers):
                if hyper_idx in progress['outer_folds'][str(outer_fold_idx)]['completed_inner_folds'][inner_fold_key]['completed_hyperparams']:
                    print(f"Hyperparameter {hyper_idx+1}/{len(hypers)} already completed. Skipping.")
                    pbar.update(1)
                    continue

                print(f"\nTraining with Hyperparameter {hyper_idx+1}: {hyper}")

                try:
                    model = model_class(
                        num_metadata_features=len(metadata_features),
                        num_classes=len(np.unique(y)),
                        pretrained=True
                    ).to(device)

                    model.set_fine_tuning(hyper['fine_tune'])
                    optimizer_name = hyper['optimizer']

                    if optimizer_name == 'AdamW':
                        optimizer = torch.optim.AdamW(
                            filter(lambda p: p.requires_grad, model.parameters()),
                            lr=hyper['lr'],
                            weight_decay=hyper.get('weight_decay', 1e-4)
                        )
                    elif optimizer_name == 'SGD':
                        optimizer = torch.optim.SGD(
                            filter(lambda p: p.requires_grad, model.parameters()),
                            lr=hyper['lr'],
                            weight_decay=hyper.get('weight_decay', 1e-4),
                            momentum=0.9
                        )
                    else:
                        raise ValueError(f"Unsupported optimizer: {optimizer_name}")

                    total_steps = len(X_train) // hyper['batch_size'] * hyper.get('epochs', 30)
                    scheduler = get_linear_schedule_with_warmup(
                        optimizer,
                        num_warmup_steps=int(0.1 * total_steps),
                        num_training_steps=total_steps
                    )

                    # Define loss function with class weights
                    class_counts = np.bincount(y_train)
                    class_weights = 1. / torch.tensor(class_counts, dtype=torch.float32).to(device)
                    criterion = nn.CrossEntropyLoss(weight=class_weights)


                    train_dataset = SkinCancerDataset(
                        dataframe=X_train.assign(dx_encoded=y_train),
                        image_dirs=image_dirs,
                        feature_extractor=feature_extractor,
                        metadata_features=metadata_features,
                        transform=transform_train
                    )
                    val_dataset = SkinCancerDataset(
                        dataframe=X_val.assign(dx_encoded=y_val),
                        image_dirs=image_dirs,
                        feature_extractor=feature_extractor,
                        metadata_features=metadata_features,
                        transform=transform_val
                    )

                    train_loader = DataLoader(
                        train_dataset,
                        batch_size=hyper['batch_size'],
                        shuffle=True,
                        num_workers=4,
                        pin_memory=True
                    )
                    val_loader = DataLoader(
                        val_dataset,
                        batch_size=hyper['batch_size'],
                        shuffle=False,
                        num_workers=4,
                        pin_memory=True
                    )


                    scaler = GradScaler()

                    best_val_acc_inner = 0.0
                    patience = 5
                    epochs_no_improve = 0

                    # Training loop with early stopping
                    for epoch in range(hyper.get('epochs', 30)):
                        epoch_start_time = time.time()
                        train_loss, train_acc = train_epoch(
                            model=model,
                            dataloader=train_loader,
                            optimizer=optimizer,
                            scheduler=scheduler,
                            device=device,
                            loss_fn=criterion,
                            scaler=scaler,
                            input_types=['image', 'metadata']
                        )

                        val_loss, val_acc = eval_epoch(
                            model=model,
                            dataloader=val_loader,
                            device=device,
                            loss_fn=criterion,
                            input_types=['image', 'metadata']
                        )

                        epoch_end_time = time.time()


                        print(f"Outer Fold {outer_fold_idx}, Inner Fold {inner_fold_idx}, Epoch {epoch+1}/{hyper.get('epochs', 30)} - "
                              f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc*100:.2f}%, "
                              f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc*100:.2f}%, "
                              f"Time: {epoch_end_time - epoch_start_time:.2f}s")

                        # Early Stopping
                        if val_acc > best_val_acc_inner:
                            best_val_acc_inner = val_acc
                            epochs_no_improve = 0

                            checkpoint = {
                                'model_state_dict': model.state_dict(),
                                'val_acc': val_acc
                            }
                            checkpoint_filename = f'outer{outer_fold_idx}_inner{inner_fold_idx}_hyper{hyper_idx+1}.pth'
                            checkpoint_path = os.path.join(NESTED_CHECKPOINT_DIR, checkpoint_filename)
                            torch.save(checkpoint, checkpoint_path)
                            print(f"Checkpoint saved at {checkpoint_path} with Val Acc: {val_acc:.4f}")
                        else:
                            epochs_no_improve += 1

                        if epochs_no_improve >= patience:
                            print(f"Early stopping at epoch {epoch+1} for Hyperparameter {hyper_idx+1}")
                            break


                    progress['outer_folds'][str(outer_fold_idx)]['completed_inner_folds'][inner_fold_key]['completed_hyperparams'].append(hyper_idx)

                    with open(progress_save_path, 'w') as f:
                        json.dump(progress, f, indent=4)

                    pbar.update(1)

                except Exception as e:
                    import traceback
                    print(f"Error during training with Hyperparameter {hyper_idx+1}: {e}")
                    print(traceback.format_exc())

                    with open(log_file, "a") as log:
                        log.write(f"Error during training with Hyperparameter {hyper_idx+1}: {e}\n")
                        log.write(traceback.format_exc())

                    continue


            outer_fold_results = progress['outer_folds'][str(outer_fold_idx)]['completed_inner_folds']

            hyper_acc = defaultdict(list)
            for inner_fold, inner_data in outer_fold_results.items():
                for hyper_idx in inner_data['completed_hyperparams']:

                    checkpoint_filename = f'outer{outer_fold_idx}_inner{inner_fold_idx}_hyper{hyper_idx+1}.pth'
                    checkpoint_path = os.path.join(NESTED_CHECKPOINT_DIR, checkpoint_filename)
                    if os.path.exists(checkpoint_path):
                        checkpoint = torch.load(checkpoint_path, map_location=device)
                        val_acc = checkpoint.get('val_acc', 0.0)
                        hyper_acc[hyper_idx].append(val_acc)
                    else:
                        print(f"Checkpoint {checkpoint_path} not found. Skipping.")

            # Aggregate validation accuracy
            if hyper_acc:

                avg_hyper_acc = {k: np.mean(v) for k, v in hyper_acc.items()}
                best_hyper_idx = max(avg_hyper_acc, key=avg_hyper_acc.get)
                best_hyperparams = hypers[best_hyper_idx]
                best_score = avg_hyper_acc[best_hyper_idx]
            else:
                best_hyperparams = {}
                best_score = 0.0


            progress['outer_folds'][str(outer_fold_idx)]['best_hyperparams'] = best_hyperparams
            progress['outer_folds'][str(outer_fold_idx)]['best_score'] = best_score

            progress['completed_outer_folds'].append(outer_fold_idx)

            with open(progress_save_path, 'w') as f:
                json.dump(progress, f, indent=4)


            best_hyperparams_checkpoint = os.path.join(NESTED_CHECKPOINT_DIR, f'outer_fold{outer_fold_idx}_best_hyperparams.json')
            with open(best_hyperparams_checkpoint, 'w') as f:
                json.dump(best_hyperparams, f, indent=4)
            print(f"Best hyperparameters for Outer Fold {outer_fold_idx} saved at {best_hyperparams_checkpoint}")


            with open(log_file, "a") as log:
                log.write(f"Outer Fold {outer_fold_idx} completed.\n")
                log.write(f"Best Params: {best_hyperparams}, Best Val Accuracy: {best_score:.2f}\n")
                log.write("--------------------------------------------------\n")

            print(f"Fold {outer_fold_idx}: Best Params: {best_hyperparams}, Best Val Accuracy: {best_score:.2f}")


            pbar.update(1)
            pbar.set_postfix({
                'Outer Fold': outer_fold_idx,
                'Status': 'Completed',
                'Best Val Acc': best_score
            })

    pbar.close()
    return results

## Feature-Level Fusion Models


### Data Preparation


In [ ]:
# Data Cleaning and Encoding for Metadata
metadata['age'].fillna(metadata['age'].median(), inplace=True)
metadata = metadata.infer_objects(copy=False)
metadata['sex'].fillna('unknown', inplace=True)
metadata = metadata.infer_objects(copy=False)
metadata['localization'].fillna('unknown', inplace=True)
metadata = metadata.infer_objects(copy=False)

# Encoding Categorical Features
le_dx = LabelEncoder()
metadata['dx_encoded'] = le_dx.fit_transform(metadata['dx'])

le_sex = LabelEncoder()
metadata['sex_encoded'] = le_sex.fit_transform(metadata['sex'])

le_loc = LabelEncoder()
metadata['localization_encoded'] = le_loc.fit_transform(metadata['localization'])

# Normalize Numerical Features
scaler = StandardScaler()
metadata['age'] = scaler.fit_transform(metadata[['age']])

# Define Feature Columns
metadata_features = ['age', 'sex_encoded', 'localization_encoded']

# Define Number of Classes
num_classes = len(le_dx.classes_)
print(f"Number of classes: {num_classes}")
print(le_dx.classes_)

# Define feature extractor
feature_extractor = models.resnet34(pretrained=ResNet34_Weights.IMAGENET1K_V1)

<ipython-input-13-57a2924149b0>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  metadata['age'].fillna(metadata['age'].median(), inplace=True)
<ipython-input-13-57a2924149b0>:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=Tr

Number of classes: 7
['akiec' 'bcc' 'bkl' 'df' 'mel' 'nv' 'vasc']


Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 202MB/s]


In [ ]:
metadata.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,dx_encoded,sex_encoded,localization_encoded
0,HAM_0000118,ISIC_0027419,bkl,histo,1.663522,male,scalp,2,1,11
1,HAM_0000118,ISIC_0025030,bkl,histo,1.663522,male,scalp,2,1,11
2,HAM_0002730,ISIC_0026769,bkl,histo,1.663522,male,scalp,2,1,11
3,HAM_0002730,ISIC_0025661,bkl,histo,1.663522,male,scalp,2,1,11
4,HAM_0001466,ISIC_0031633,bkl,histo,1.368014,male,ear,2,1,4


### Dataset Class


In [ ]:
class SkinCancerDataset(Dataset):
    """
      A custom PyTorch Dataset for handling multimodal data consisting of images and metadata for skin cancer classification.

      Attributes:
          dataframe (pd.DataFrame): DataFrame containing metadata and labels for the dataset.
          image_dirs (list): List of directories where images are stored.
          feature_extractor (ViTFeatureExtractor): Feature extractor for processing images.
          transform (callable, optional): Optional transform to be applied on an image.
          metadata_features (list): List of metadata feature column names to be used.

      Methods:
          __init__(dataframe, image_dirs, feature_extractor, metadata_features, transform=None):
              Initializes the dataset with the given parameters.

          __len__():
              Returns the number of samples in the dataset.

          __getitem__(idx):
              Retrieves the image, metadata, and label for a given index.
    """
    def __init__(self, dataframe, image_dirs, feature_extractor, metadata_features, transform=None):
        """
        Initializes the SkinCancerMultimodalDataset.
        """
        self.dataframe = dataframe
        self.image_dirs = image_dirs
        self.feature_extractor = feature_extractor
        self.transform = transform
        self.metadata_features = metadata_features

    def __len__(self):
        """
        Returns the number of samples in the dataset.
        """
        return len(self.dataframe)

    def __getitem__(self, idx):
        """
        Retrieves the image, metadata, and label for a given index.
        """
        row = self.dataframe.iloc[idx]
        image_id = row['image_id']
        # Search for the image in the provided directories
        for directory in self.image_dirs:
            image_path = os.path.join(directory, f"{image_id}.jpg")
            if os.path.exists(image_path):
                break
        image = Image.open(image_path).convert("RGB")
        if self.transform:
            image = self.transform(image=np.array(image))['image']
        metadata = torch.tensor(row[self.metadata_features].values.astype(np.float32))
        label = torch.tensor(row['dx_encoded'])
        return image, metadata, label

### Transform Pipeline

In [ ]:
train_transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.Rotate(limit=20, p=0.5),
    A.Resize(224, 224),
    #ImageNet mean and standard dev.
    A.Normalize(mean=(0.485, 0.456, 0.406),
                std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])

val_test_transform = A.Compose([
    A.Resize(224, 224),
    A.Normalize(mean=(0.485, 0.456, 0.406),
                std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])

### Train/Eval Loop

In [ ]:
def train_epoch(model, dataloader, optimizer, scheduler, device, loss_fn, scaler, input_types=['image', 'metadata']):
    """
    Trains a model for one epoch, handling various input types with mixed precision.

    Args:
        model (nn.Module): The neural network model to train.
        dataloader (DataLoader): DataLoader for the training data.
        optimizer (torch.optim.Optimizer): Optimizer for updating model parameters.
        scheduler (torch.optim.lr_scheduler): Learning rate scheduler.
        device (torch.device): Device to perform training on (CPU or GPU).
        loss_fn (nn.Module): Loss function, e.g., CrossEntropyLoss.
        scaler (GradScaler): Gradient scaler for mixed precision.
        input_types (list): List specifying the types of inputs the model expects (e.g., ['image'], ['image', 'metadata']).

    Returns:
        tuple: (epoch_loss, epoch_acc)
    """
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for batch_idx, data in enumerate(tqdm(dataloader, desc="Training", mininterval=5.0)):
        # Unpack and move data to device
        if 'metadata' in input_types:
            images, metadata, labels = data
            images = images.to(device, non_blocking=True)
            metadata = metadata.to(device, non_blocking=True)
        else:
            images, labels = data
            images = images.to(device, non_blocking=True)

        labels = labels.to(device, non_blocking=True)

        # Zero gradients
        optimizer.zero_grad()

        # Forward pass with autocast
        with autocast(device_type=device.type):
            if 'metadata' in input_types:
                outputs = model(images, metadata)
            else:
                outputs = model(images)
            loss = loss_fn(outputs, labels)

        # Backward pass with scaled gradients
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()

        # Accumulate loss and accuracy
        running_loss += loss.item() * labels.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

def eval_epoch(model, dataloader, device, loss_fn, input_types=['image', 'metadata']):
    """
    Evaluates a model on a validation or test set, handling various input types with mixed precision.

    Args:
        model (nn.Module): The neural network model to evaluate.
        dataloader (DataLoader): DataLoader for the evaluation data.
        device (torch.device): Device to perform evaluation on (CPU or GPU).
        loss_fn (nn.Module): Loss function, e.g., CrossEntropyLoss.
        input_types (list): List specifying the types of inputs the model expects (e.g., ['image'], ['image', 'metadata']).

    Returns:
        tuple: (epoch_loss, epoch_acc)
    """
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for batch_idx, data in enumerate(tqdm(dataloader, desc="Evaluating", mininterval=5.0)):
            # Unpack and move data to device
            if 'metadata' in input_types:
                images, metadata, labels = data
                images = images.to(device, non_blocking=True)
                metadata = metadata.to(device, non_blocking=True)
            else:
                images, labels = data
                images = images.to(device, non_blocking=True)

            labels = labels.to(device, non_blocking=True)

            # Forward pass with autocast
            with autocast(device_type=device.type):
                if 'metadata' in input_types:
                    outputs = model(images, metadata)
                else:
                    outputs = model(images)
                loss = loss_fn(outputs, labels)

            # Accumulate loss and accuracy
            running_loss += loss.item() * labels.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

### ResNet-Fusion Model

In [ ]:
# Implementation

from torchvision.models import ResNet34_Weights

class EarlyFusionModel(nn.Module):
    """
    A neural network model for early fusion of image and metadata features using ResNet34.
    """
    def __init__(self, num_metadata_features, num_classes, pretrained=True):
        super(EarlyFusionModel, self).__init__()
        # Load pre-trained ResNet34
        self.resnet = models.resnet34(weights=ResNet34_Weights.IMAGENET1K_V1)
        # Remove the final fully connected layer
        self.resnet = nn.Sequential(*list(self.resnet.children())[:-1])  =
        # ResNet34's final feature size
        resnet_output_size = 512

        # Metadata processing
        self.metadata_fc = nn.Sequential(
            nn.Linear(num_metadata_features, 128),
            nn.ReLU(),
            nn.Dropout(0.1)
        )

        # Classifier
        self.classifier = nn.Linear(resnet_output_size + 128, num_classes)

    def forward(self, image, metadata):
        # Extract image features
        image_features = self.resnet(image)
        image_features = image_features.view(image_features.size(0), -1)

        # Process metadata features
        metadata_features = self.metadata_fc(metadata)

        # Combine features
        combined_features = torch.cat((image_features, metadata_features), dim=1)
        logits = self.classifier(combined_features)
        return logits

    def set_fine_tuning(self, fine_tune='none'):
        """
        Sets the fine-tuning strategy by freezing or unfreezing ResNet layers.

        Args:
            fine_tune (str): 'none' for feature extraction, 'all' for full fine-tuning.
        """
        if fine_tune == 'none':
            # Freeze all ResNet34 layers
            for param in self.resnet.parameters():
                param.requires_grad = False
            print("ResNet34 layers frozen. Only classifier and metadata layers will be trained.")
        elif fine_tune == 'all':
            # Unfreeze all ResNet34 layers
            for param in self.resnet.parameters():
                param.requires_grad = True
            print("ResNet34 layers unfrozen. All layers will be trained.")
        else:
            raise ValueError(f"Unsupported fine_tune option: {fine_tune}")

#### Single Run

In [ ]:
# Split Data into Train, Validation, and Test Sets (from scratch)

X = metadata.drop(columns=['dx_encoded']).values
y = metadata['dx_encoded'].values

outer_skf = customStratifiedKFold(n_split=5, shuffle=True, random_state=42)

# Get the first fold as the test set
for fold, (train_val_idx, test_idx) in enumerate(outer_skf.split(X, y)):
    if fold == 0:
        X_train_val, X_test = X[train_val_idx], X[test_idx]
        y_train_val, y_test = y[train_val_idx], y[test_idx]
        break

inner_skf = customStratifiedKFold(n_split=10, shuffle=True, random_state=42)


for fold, (train_idx, val_idx) in enumerate(inner_skf.split(X_train_val, y_train_val)):
    if fold == 0:
        X_train, X_val = X_train_val[train_idx], X_train_val[val_idx]
        y_train, y_val = y_train_val[train_idx], y_train_val[val_idx]
        break

train_df = metadata.iloc[train_val_idx[train_idx]]
val_df = metadata.iloc[train_val_idx[val_idx]]
test_df = metadata.iloc[test_idx]

print(f"Training samples: {len(train_df)}")
print(f"Validation samples: {len(val_df)}")
print(f"Testing samples: {len(test_df)}")

In [ ]:
def plot_class_distribution(df, label_column, title):
    """
    Plots the class distribution of a given DataFrame.

    Args:
        df (pd.DataFrame): The DataFrame containing the data.
        label_column (str): The column name for class labels.
        title (str): The title for the plot.
    """
    plt.figure(figsize=(10, 6))
    sns.countplot(x=label_column, data=df, order=df[label_column].value_counts().index)
    plt.title(title)
    plt.xlabel('Class')
    plt.ylabel('Frequency')
    plt.xticks(rotation=45)
    plt.show()

plot_class_distribution(train_df, 'dx', 'Training Set Class Distribution')
plot_class_distribution(val_df, 'dx', 'Validation Set Class Distribution')
plot_class_distribution(test_df, 'dx', 'Test Set Class Distribution')

In [ ]:
# Get class counts for the training, validation, and test sets
_, train_class_counts = np.unique(y_train, return_counts=True)
_, val_class_counts = np.unique(y_val, return_counts=True)
_, test_class_counts = np.unique(y_test, return_counts=True)

train_class_counts_list = list(train_class_counts)
val_class_counts_list = list(val_class_counts)
test_class_counts_list = list(test_class_counts)

# Print class counts for each set
print(f"Training set class counts: {train_class_counts_list}")
print(f"Validation set class counts: {val_class_counts_list}")
print(f"Test set class counts: {test_class_counts_list}")

In [ ]:
# Creating Multimodal Datasets and DataLoaders
train_dataset = SkinCancerDataset(train_df, IMAGE_DIRS, feature_extractor, metadata_features, transform=train_transform)
val_dataset = SkinCancerDataset(val_df, IMAGE_DIRS, feature_extractor, metadata_features, transform=val_test_transform)
test_dataset = SkinCancerDataset(test_df, IMAGE_DIRS, feature_extractor, metadata_features, transform=val_test_transform)

BATCH_SIZE = 128

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

print(f"Number of training batches: {len(train_loader)}")
print(f"Number of validation batches: {len(val_loader)}")
print(f"Number of testing batches: {len(test_loader)}")


In [ ]:
# Initialize Early Fusion Model
num_metadata_features = len(metadata_features)
model_early_fusion = EarlyFusionModel(num_metadata_features=num_metadata_features, num_classes=num_classes, pretrained=True)
model_early_fusion.to(DEVICE)

# Define Optimizer and Scheduler
EPOCHS = 10
optimizer_early = torch.optim.AdamW(model_early_fusion.parameters(), lr=1e-4)
scheduler_early = get_linear_schedule_with_warmup(
    optimizer_early,
    num_warmup_steps=int(0.1 * len(train_loader) * EPOCHS),
    num_training_steps=len(train_loader) * EPOCHS
)

In [ ]:
# Weighted class distribution
class_counts_tensor = torch.tensor(train_class_counts_list, dtype=torch.float32)
class_weights = 1.0 / class_counts_tensor
class_weights = class_weights / class_weights.sum()
class_weights = class_weights.to(DEVICE)

# Define the train loss function
criterion_train = nn.CrossEntropyLoss(weight=class_weights)

# Define the eval loss function
criterion_eval = nn.CrossEntropyLoss()

# Training Loop
best_val_acc = 0.0

for epoch in range(EPOCHS):
    print(f"\nEpoch {epoch+1}/{EPOCHS}")
    print("-" * 10)

    # Training Phase
    train_loss, train_acc = train_epoch(
        model=model_early_fusion,
        dataloader=train_loader,
        optimizer=optimizer_early,
        scheduler=scheduler_early,
        device=DEVICE,
        loss_fn=criterion_train,
        input_types=['image', 'metadata']
    )

    print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc*100:.2f}%\n")

    # Evaluation Phase
    val_loss, val_acc = eval_epoch(
        model=model_early_fusion,
        dataloader=val_loader,
        device=DEVICE,
        loss_fn=criterion_eval,
        input_types=['image', 'metadata']
    )

    print(f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc*100:.2f}%")

    # Checkpointing
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        os.makedirs('models/early_fusion', exist_ok=True)
        torch.save(model_early_fusion.state_dict(), 'models/early_fusion/best_model.pth')
        print("Model checkpoint saved.")

In [ ]:
def plot_confusion_matrix(model, dataloader, device, class_names):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, metadata, labels in dataloader:
            images = images.to(device)
            metadata = metadata.to(device)
            labels = labels.to(device)

            outputs = model(images, metadata)
            _, preds = torch.max(outputs, 1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    cm = confusion_matrix(all_labels, all_preds)
    plt.figure(figsize=(10,8))
    sns.heatmap(cm, annot=True, fmt='d', xticklabels=class_names, yticklabels=class_names, cmap='Blues')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title('Confusion Matrix')
    plt.show()

class_names = ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']

plot_confusion_matrix(model_early_fusion, val_loader, DEVICE, class_names)

#### Nested CV

In [ ]:
X = metadata.drop(columns=['dx_encoded']).values
y = metadata['dx_encoded'].values

# Initialize the custom stratified K-Fold for splitting 50% of the data for hyperparameter tuning
tuning_kfold = customStratifiedKFold(n_split=2, shuffle=True, random_state=42)
splits = list(tuning_kfold.split(X, y))
train_main_idx, test_tuning_idx = splits[0]

columns = ['lesion_id', 'image_id', 'dx', 'dx_type', 'age','sex','localization','sex_encoded','localization_encoded']
X_tuning = pd.DataFrame(X[test_tuning_idx], columns=columns)
y_tuning = y[test_tuning_idx]

print("Columns in X_tuning:", X_tuning.columns.tolist())
print("Shape of X_tuning:", X_tuning.shape)
print("Shape of y_tuning:", y_tuning.shape)

X_tuning.head()

Columns in X_tuning: ['lesion_id', 'image_id', 'dx', 'dx_type', 'age', 'sex', 'localization', 'sex_encoded', 'localization_encoded']
Shape of X_tuning: (5010, 9)
Shape of y_tuning: (5010,)


,lesion_id,image_id,dx,dx_type,age,sex,localization,sex_encoded,localization_encoded
0,HAM_0000597,ISIC_0031822,nv,histo,-0.996054,female,abdomen,0,0
1,HAM_0002995,ISIC_0028886,nv,follow_up,0.481489,female,back,0,2
2,HAM_0006846,ISIC_0027199,nv,follow_up,-0.405037,male,lower extremity,1,9
3,HAM_0002348,ISIC_0025240,nv,histo,-0.405037,male,abdomen,1,0
4,HAM_0002163,ISIC_0030250,nv,follow_up,-0.109528,male,trunk,1,12


In [ ]:
# Create directories if they don't exist
for directory in [NESTED_RESULT_SAVE_DIR, NESTED_MODEL_SAVE_DIR, NESTED_CHECKPOINT_DIR,
                FOLD20_RESULT_SAVE_DIR, FOLD20_MODEL_SAVE_DIR, FOLD20_CHECKPOINT_DIR]:
    os.makedirs(directory, exist_ok=True)

# Updated Hyperparameter Sets with Fine-Tuning Strategy
hypers = [
    {
        'lr': 1e-5,
        'batch_size': 64,
        'weight_decay': 1e-4,
        'optimizer': 'AdamW',
        'epochs': 30,
        'fine_tune': 'none'
    }, # Conservative (Stable Learning, Feature Extraction)

    {
        'lr': 5e-4,
        'batch_size': 128,
        'weight_decay': 1e-4,
        'optimizer': 'AdamW',
        'epochs': 30,
        'fine_tune': 'none'
    }, # Aggressive (Fast Learning, Feature Extraction)

    {
        'lr': 1e-4,
        'batch_size': 256,
        'weight_decay': 5e-4,
        'optimizer': 'AdamW',
        'epochs': 30,
        'fine_tune': 'all'
    }, # Middle-Ground (Balance, Full Fine-Tuning)

    {
        'lr': 1e-4,
        'batch_size': 128,
        'weight_decay': 1e-4,
        'optimizer': 'SGD',
        'epochs': 30,
        'fine_tune': 'all'
    }, # Different Optimizer, Full Fine-Tuning
]

metadata_features = ['age', 'sex_encoded', 'localization_encoded']
feature_extractor = models.resnet34(pretrained=ResNet34_Weights.IMAGENET1K_V1)

nested_cv_params = {
    'model_class': EarlyFusionModel,
    'X': X_tuning,
    'y': y_tuning,
    'outer_splits': 5,
    'inner_splits': 3,
    'hypers': hypers,
    'metadata_features': metadata_features,
    'image_dirs': IMAGE_DIRS,
    'feature_extractor': feature_extractor,
    'transform_train': train_transform,
    'transform_val': val_test_transform,
    'device': DEVICE,
    'log_file': os.path.join(NESTED_CHECKPOINT_DIR, 'nested_cv_log.txt')
}

nested_results = cross_validate(**nested_cv_params)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 193MB/s]


Initialized new progress tracking.
Created and saved new data splits.


Nested Cross-Validation Progress:   0%|          | 0/65 [00:00<?, ?combo/s]


=== Starting Outer Fold 1/5 ===

--- Outer Fold 1, Inner Fold 1/3 ---

Training with Hyperparameter 1: {'lr': 1e-05, 'batch_size': 64, 'weight_decay': 0.0001, 'optimizer': 'AdamW', 'epochs': 30, 'fine_tune': 'none'}


<ipython-input-10-bff09d76f60b>:203: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


...

Outer Fold 5, Inner Fold 3, Epoch 29/30 - Train Loss: 0.9588, Train Acc: 65.70%, Val Loss: 1.1759, Val Acc: 62.62%, Time: 11.03s



Training: 100%|██████████| 21/21 [00:07<00:00,  2.85it/s]

Evaluating: 100%|██████████| 11/11 [00:03<00:00,  3.15it/s]
Nested Cross-Validation Progress: 74combo [4:38:11, 245.76s/combo, Outer Fold=5, Status=Completed, Best Val Acc=0.811]<ipython-input-10-bff09d76f60b>:290: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any u

Outer Fold 5, Inner Fold 3, Epoch 30/30 - Train Loss: 0.9592, Train Acc: 65.70%, Val Loss: 1.1792, Val Acc: 62.55%, Time: 10.88s


Nested Cross-Validation Progress: 75combo [4:38:12, 222.57s/combo, Outer Fold=5, Status=Completed, Best Val Acc=0.824]

Best hyperparameters for Outer Fold 5 saved at drive/MyDrive/fusion_Checkpoints/nested_cv/outer_fold5_best_hyperparams.json
Fold 5: Best Params: {'lr': 0.0001, 'batch_size': 256, 'weight_decay': 0.0005, 'optimizer': 'AdamW', 'epochs': 30, 'fine_tune': 'all'}, Best Val Accuracy: 0.82


In [ ]:
import re

log_file_path = NESTED_CHECKPOINT_DIR+'/nested_cv_log.txt'

hyper_results = defaultdict(list)
pattern = re.compile(r"Best Params: (.+), Best Val Accuracy: ([\d.]+)")

with open(log_file_path, 'r') as log_file:
    for line in log_file:
        match = pattern.search(line)
        if match:

            hyperparams_str = match.group(1)
            val_acc = float(match.group(2))
            hyperparams = eval(hyperparams_str)
            hyperparams_key = frozenset(hyperparams.items())
            hyper_results[hyperparams_key].append(val_acc)

# Calculate the average validation accuracy for each hyperparameter configuration
avg_hyper_results = {k: sum(v) / len(v) for k, v in hyper_results.items()}

# Find the best hyperparameter configuration
best_hyperparams = max(avg_hyper_results, key=avg_hyper_results.get)
best_score = avg_hyper_results[best_hyperparams]

# Convert frozenset back to a dictionary for readability
best_hyperparams_dict = dict(best_hyperparams)

print(f"Best Hyperparameters: {best_hyperparams_dict}")
print(f"Best Average Validation Accuracy: {best_score:.4f}")

Best Hyperparameters: {'epochs': 30, 'fine_tune': 'all', 'weight_decay': 0.0005, 'optimizer': 'AdamW', 'lr': 0.0001, 'batch_size': 256}
Best Average Validation Accuracy: 0.8220


#### Independent 20-fold CV

In [ ]:
# Create a Training Function for Each Fold
def train_model(X_train, y_train, X_test, y_test, device, params, fold):
    """
    Trains the model on the training set and evaluates on the test set.

    Args:
        X_train (pd.DataFrame): Training DataFrame.
        y_train (pd.Series or np.ndarray): Training labels.
        X_test (pd.DataFrame): Test DataFrame.
        y_test (pd.Series or np.ndarray): Test labels.
        device (torch.device): Device to perform training on.
        params (dict): Hyperparameters for training.
        fold (int): Current fold index.

    Returns:
        tuple: (trained model, best validation accuracy)
    """
    # Initialize the model
    model = EarlyFusionModel(
        num_metadata_features=len(metadata_features),
        num_classes=len(np.unique(y_train)),
        pretrained=True
    )
    model.to(device)

    # Set fine-tuning strategy
    model.set_fine_tuning(fine_tune=params['fine_tune'])

    # Define optimizer based on the provided hyperparameters
    if params['optimizer'] == 'AdamW':
        optimizer = torch.optim.AdamW(model.parameters(), lr=params['lr'], weight_decay=params['weight_decay'])
    elif params['optimizer'] == 'SGD':
        optimizer = torch.optim.SGD(model.parameters(), lr=params['lr'], weight_decay=params['weight_decay'], momentum=0.9)
    else:
        raise ValueError(f"Unsupported optimizer: {params['optimizer']}")

    # Define learning rate scheduler
    total_steps = len(X_train) // params['batch_size'] * params['epochs']
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=int(0.1 * total_steps),
        num_training_steps=total_steps
    )

    # Define the loss function with optional class weights (based on class imbalance)
    class_counts = np.bincount(y_train)
    class_weights = 1. / torch.tensor(class_counts, dtype=torch.float32).to(device)
    criterion = nn.CrossEntropyLoss(weight=class_weights)

    # Create Datasets and DataLoaders
    train_dataset = SkinCancerDataset(
        dataframe=X_train.assign(dx_encoded=y_train),
        image_dirs=IMAGE_DIRS,
        feature_extractor=feature_extractor,
        metadata_features=metadata_features,
        transform=train_transform
    )
    test_dataset = SkinCancerDataset(
        dataframe=X_test.assign(dx_encoded=y_test),
        image_dirs=IMAGE_DIRS,
        feature_extractor=feature_extractor,
        metadata_features=metadata_features,
        transform=val_test_transform
    )

    train_loader = DataLoader(
        train_dataset,
        batch_size=params['batch_size'],
        shuffle=True,
        num_workers=4,
        pin_memory=True
    )
    test_loader = DataLoader(
        test_dataset,
        batch_size=params['batch_size'],
        shuffle=False,
        num_workers=4,
        pin_memory=True
    )

    # Initialize GradScaler for Mixed Precision
    scaler = GradScaler()

    best_val_acc = 0
    patience_counter = 0

    # Training loop with early stopping
    for epoch in range(params['epochs']):
        epoch_start_time = time.time()
        train_loss, train_acc = train_epoch(
            model=model,
            dataloader=train_loader,
            optimizer=optimizer,
            scheduler=scheduler,
            device=device,
            loss_fn=criterion,
            scaler=scaler,
            input_types=['image', 'metadata']
        )

        val_loss, val_acc = eval_epoch(
            model=model,
            dataloader=test_loader,
            device=device,
            loss_fn=criterion,
            input_types=['image', 'metadata']
        )

        epoch_end_time = time.time()

        print(f"Fold {fold+1}, Epoch {epoch+1}/{params['epochs']} - "
              f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc*100:.2f}%, "
              f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc*100:.2f}%, "
              f"Time: {epoch_end_time - epoch_start_time:.2f}s")

        # Early stopping
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            patience_counter = 0

            best_model_state = model.state_dict()
            best_model_path = os.path.join(FOLD20_MODEL_SAVE_DIR, f'best_model_fold{fold+1}.pth')
            torch.save(best_model_state, best_model_path)
            print(f"New best model saved from Fold {fold+1} with validation accuracy: {val_acc:.4f}")
        else:
            patience_counter += 1

        PATIENCE = 5
        if patience_counter >= PATIENCE:
            print("Early stopping triggered.")
            break

    model.eval()
    predictions = []
    actuals = []
    with torch.no_grad():
        for images, metadata, labels in tqdm(test_loader, desc="Saving Test Predictions"):
            images = images.to(device, non_blocking=True)
            metadata = metadata.to(device, non_blocking=True)
            outputs = model(images, metadata)
            _, predicted = torch.max(outputs, 1)
            predictions.extend(predicted.cpu().numpy())
            actuals.extend(labels.cpu().numpy())

    test_result_df = pd.DataFrame({
        'Prediction': predictions,
        'Actual': actuals
    })
    test_file_name = f"fold{fold+1}_test.csv"
    test_result_df.to_csv(os.path.join(FOLD20_RESULT_SAVE_DIR, test_file_name), index=False)
    print(f"Fold {fold+1} test predictions saved to {test_file_name}")

    train_predictions = []
    train_actuals = []
    with torch.no_grad():
        for images, metadata, labels in tqdm(train_loader, desc="Saving Train Predictions"):
            images = images.to(device, non_blocking=True)
            metadata = metadata.to(device, non_blocking=True)
            outputs = model(images, metadata)
            _, predicted = torch.max(outputs, 1)
            train_predictions.extend(predicted.cpu().numpy())
            train_actuals.extend(labels.cpu().numpy())

    train_result_df = pd.DataFrame({
        'Prediction': train_predictions,
        'Actual': train_actuals
    })
    train_file_name = f"fold{fold+1}_train.csv"
    train_result_df.to_csv(os.path.join(FOLD20_RESULT_SAVE_DIR, train_file_name), index=False)
    print(f"Fold {fold+1} train predictions saved to {train_file_name}")

    return model, best_val_acc

In [ ]:
# Main 20-fold Cross-Validation with stratifiedKFold

hyperparams = {'epochs': 30,
               'fine_tune': 'all',
               'weight_decay': 0.0005,
               'optimizer': 'AdamW',
               'lr': 0.0001,
               'batch_size': 256}

X = metadata.drop(columns=['dx_encoded'])
y = metadata['dx_encoded'].values

final_splits = 20
skf = customStratifiedKFold(n_split=final_splits, shuffle=True, random_state=42)
indices = list(skf.split(X.values, y))

# Function to Save Indices
def save_indices(indices, path):
    with open(path, 'wb') as f:
        pickle.dump(indices, f)

# Function to Load Indices
def load_indices(path):
    with open(path, 'rb') as f:
        return pickle.load(f)


if os.path.exists(INDICES_SAVE_PATH):
    indices = load_indices(INDICES_SAVE_PATH)
else:
    skf = customStratifiedKFold(n_split=final_splits, shuffle=True, random_state=42)
    indices = list(skf.split(X.values, y))
    save_indices(indices, INDICES_SAVE_PATH)

completed_folds = 0
for fold in range(final_splits):
    train_file = f"fold{fold+1}_train.csv"
    test_file = f"fold{fold+1}_test.csv"
    if os.path.exists(os.path.join(FOLD20_RESULT_SAVE_DIR, train_file)) and os.path.exists(os.path.join(FOLD20_RESULT_SAVE_DIR, test_file)):
        completed_folds += 1
    else:
        break

start_fold = completed_folds
best_model_acc = 0.0

for fold_idx in range(start_fold, final_splits):
    train_idx, test_idx = indices[fold_idx]
    print(f"\n=== Starting Fold {fold_idx+1}/{final_splits} ===")

    X_train = X.iloc[train_idx].reset_index(drop=True)
    y_train = y[train_idx]
    X_test = X.iloc[test_idx].reset_index(drop=True)
    y_test = y[test_idx]

    model, val_acc = train_model(X_train, y_train, X_test, y_test, DEVICE, hyperparams, fold_idx)

    if val_acc > best_model_acc:
        best_model_acc = val_acc
        best_model_state_dict = model.state_dict()

        best_model_path = os.path.join(FOLD20_MODEL_SAVE_DIR, f'best_model_fold{fold_idx+1}.pth')
        torch.save(best_model_state_dict, best_model_path)
        print(f"New best model saved from Fold {fold_idx+1} with validation accuracy: {val_acc:.4f}")

    del model
    torch.cuda.empty_cache()

print("\nFinal Cross-Validation Completed.")
print(f"Best Validation Accuracy Achieved: {best_model_acc:.4f}")


=== Starting Fold 1/20 ===


<ipython-input-20-1ffa063bc1ac>:82: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


ResNet34 layers unfrozen. All layers will be trained.


Training:   0%|          | 0/38 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(
Evaluating: 100%|██████████| 2/2 [00:03<00:00,  1.51s/it]


Fold 1, Epoch 1/30 - Train Loss: 2.0053, Train Acc: 9.12%, Val Loss: 1.3765, Val Acc: 20.24%, Time: 29.34s
New best model saved from Fold 1 with validation accuracy: 0.2024


...

Fold 20, Epoch 24/30 - Train Loss: 0.0121, Train Acc: 99.42%, Val Loss: 0.6249, Val Acc: 88.73%, Time: 26.73s
Early stopping triggered.


Saving Test Predictions: 100%|██████████| 2/2 [00:02<00:00,  1.42s/it]


Fold 20 test predictions saved to fold20_test.csv


Saving Train Predictions: 100%|██████████| 38/38 [00:24<00:00,  1.56it/s]

Fold 20 train predictions saved to fold20_train.csv

Final Cross-Validation Completed.
Best Validation Accuracy Achieved: 0.9145
